In [4]:
import os, time, itertools, pickle
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [42]:
def leaky_relu(x, th = 0.2):
    return tf.maximum(th*x, x)


def generator(x, y, istrain = True, reuse = tf.AUTO_REUSE):
    
    with tf.variable_scope('generator', reuse = reuse):
        w_init = tf.contrib.layers.xavier_initializer()
        g_input = tf.concat([x, y], 1)
        
        fc1 = tf.nn.relu(tf.layers.dense(g_input, 128, kernel_initializer = w_init))
        fc2 = tf.layers.dense(fc1, 784, kernel_initializer = w_init)
        generated_outputs = tf.nn.tanh(fc2)
        
        return generated_outputs
    
def discriminator(x, y, isTrain = True, reuse  = tf.AUTO_REUSE):
    w_init = tf.contrib.layers.xavier_initializer()
    
    d_input = tf.concat([x, y], 1)
    
    fc1 =leaky_relu(tf.layers.dense(d_input, 128, kernel_initializer = w_init))
    fc2 = tf.layers.dense(fc1, 1, kernel_initializer = w_init)
    discriminator_outputs = tf.nn.sigmoid(fc2)
    
    return discriminator_outputs, fc2

In [43]:
x  = tf.placeholder(tf.float32, shape = (None, 784))
y  = tf.placeholder(tf.float32, shape = (None, 10))
z  = tf.placeholder(tf.float32, shape = (None, 100))
isTrain = tf.placeholder(dtype=tf.bool)

In [44]:
batch_size = 100
leraning_rate = 0.002
epochs = 100

mnist = input_data.read_data_sets("MNIST_data/", one_hot  = True)
train_set = (mnist.train.images  - 0.5) / 0.5
print(np.shape(train_set))
print(len(train_set))

train_labels = mnist.train.labels
print(len(train_labels))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
(55000, 784)
55000
55000


In [46]:
G_z = generator(z, y, isTrain)
D_real, D_real_logits = discriminator(x, y, isTrain)
D_fake, D_fake_logits = discriminator(G_z, y, isTrain, reuse = True)

D_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = D_real_logits,  
                                                                     labels = tf.ones([batch_size, 1])))

D_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = D_fake_logits,  
                                                                     labels = tf.zeros([batch_size, 1])))

G_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = D_fake_logits,  
                                                                     labels = tf.ones([batch_size, 1])))

D_loss = D_loss_real + D_loss_fake


In [47]:
sess  = tf.InteractiveSession()
tf.global_variables_initializer().run()

In [50]:
onehot = np.eye(10)

fixed_z_ = np.random.normal(0,1 (25, 100))
fixed_y_ = np.random.randint(0, 9, (25, 1))
fixed_y_ = onehot[fixed_y_.astype(np.int32)].squeeze()

def show_result(num_epoch):
    test_images = plt.subplots(size_figure_grid, size_figure_grid, figsize = (5,1))
    
    for i, j in itertools.product(range(size_figure_grid), range(size_figure_grid)):
        ax[i, j].get_xaxis().set_visible(False)
        ax[i, j].get_yaxis().set_visible(False)
        
        for k in range(size_figure_grid*size_figure_grid):
            i = k //size_figure_grid
            j = k % size_figure_grid
            ax[i, j].cla()
            ax[i,j].imshow(np.reshape(test_images[k], (28, 28)),cmap = 'gray')
            
        label = 'Epoch {0}'.format(num_epoch)
        fig.text(0.5, 0.04, label, ha = 'center')
        
        plt.show()

TypeError: 'int' object is not callable

In [ ]:
x, y, 